In [1]:
import pandas as pd
import numpy as np
import datetime
from data import LoadSales
import logging as log
#from models import *

In [2]:
data = LoadSales('teradata_sales')

In [3]:
df_i = data.transform(freq ="month",sku = 69239730)

In [6]:
df_i.to_csv("df_i.csv", sep = ";", index = False)

In [27]:
df_i = pd.read_csv("df_i.csv", sep = ";")

In [29]:
data = df_i.sales
n_test = 14
# model configs
#TODO:adding/deleting other elemnts in seasonal
cfg_list = exp_smoothing_configs(seasonal=[6,12])
len(cfg_list)


16

In [30]:
scores = grid_search(data.values, cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

 > Model[['add', False, 'mul', 6, True, True]] 139.859
 > Model[['add', False, 'add', 6, True, True]] 234.359
 > Model[['add', False, 'add', 12, True, True]] 289.897
 > Model[['add', False, 'mul', 12, True, True]] 278.539
 > Model[['mul', False, 'add', 6, True, True]] 591.302
 > Model[['mul', False, 'add', 12, True, True]] 345.942
 > Model[['add', True, 'mul', 12, True, True]] 289.183
 > Model[['add', True, 'add', 12, True, True]] 293.990
 > Model[['mul', True, 'add', 12, True, True]] 363.474
 > Model[['add', True, 'add', 6, True, True]] 237.741
 > Model[['mul', True, 'add', 6, True, True]] 415.068
 > Model[['add', True, 'mul', 6, True, True]] 178.760
done
['add', False, 'mul', 6, True, True] 139.8591939070485
['add', True, 'mul', 6, True, True] 178.7600414654915
['add', False, 'add', 6, True, True] 234.3590129907872


In [63]:
best_cfg = eval( scores.pop(0)[0] )
#forecast the data
y_hat = forecast_model(data.values, config = best_cfg, h= 12 - 1 )
#crete the dates for the forecasts
date_projection = pd.date_range(df_i.date[len(df_i)-1],periods=12+1 ,freq='MS')[1:]
#adding the forecast to the past data
data_future = pd.DataFrame()
data_future["date"] = date_projection
data_future["sales"] = y_hat
data_final = df_i.copy()
data_final["type_value"] ="a"
data_future["type_value"]= "f"
data_final = data_final.append(data_future)

In [66]:
data_final.reset_index(drop = True)

,date,sales,type_value
0,2016-07-01,429.000000,a
1,2016-08-01,331.000000,a
2,2016-09-01,336.000000,a
3,2016-10-01,412.000000,a
4,2016-11-01,595.000000,a
5,2016-12-01,1459.000000,a
6,2017-01-01,504.000000,a
7,2017-02-01,319.000000,a
8,2017-03-01,1005.000000,a
9,2017-04-01,432.000000,a


In [26]:
#https://machinelearningmastery.com/how-to-grid-search-triple-exponential-smoothing-for-time-series-forecasting-in-python/

from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas import read_csv
from numpy import array
import pandas as pd
# grid search ets models for monthly car sales
 
# one-step Holt Winter Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
	t,d,s,p,b,r = config
	# define model
	history = array(history)
	model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
	# fit model
	model_fit = model.fit(optimized=True
                         #, use_boxcox=b, remove_bias=r
                         )
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]
 
# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]
 
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = exp_smoothing_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error
 
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)
 
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores
#TODO: deleting some extra uncessary params to accelerate the process
# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
	models = list()
	# define config lists
	t_params = ['add', 'mul'
                #, None
	]
	d_params = [True, False
	]
	s_params = ['add', 'mul'
                #, None
	]
	p_params = seasonal
	b_params = [True
                #, False
	]
	r_params = [True
                #, False
	]
	# create config instances
	for t in t_params:
		for d in d_params:
			for s in s_params:
				for p in p_params:
					for b in b_params:
						for r in r_params:
							cfg = [t,d,s,p,b,r]
							models.append(cfg)
	return models


def forecast_model(history, config,h):
    t,d,s,p,b,r = config
	# define model model
    history = array(history)
    model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
    # fit model
    model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
    #make one step forecast
    yhat = model_fit.predict(start=len(history), end = len(history)+ h)
    return yhat












# link_ex = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv"
# series = pd.read_csv(link_ex, header=0, index_col=0)
# data = series.values
# # data split
# n_test = 12
# # model configs
# cfg_list = exp_smoothing_configs(seasonal=[0,6,12])
# # grid search
# scores = grid_search(data[:,0], cfg_list, n_test)
# print('done')
# # list top 3 configs
# for cfg, error in scores[:3]:
#     print(cfg, error)
